# This notebook splits eligibility from all files to inclusion and exclusion

In [1]:
import pandas as pd
import re
import xml.etree.cElementTree as et
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np

In [2]:
#trying conversion from xml to pandas dictionary
import os
import glob

# file = os.listdir("./xml/")
d = {}
eligibility = None
file = glob.glob("xml/*")
for i in range(len(file)):
    try:
        parsedXML = et.parse(file[i])
        if parsedXML is not None:
            for node in parsedXML.getroot():
                if node is not None:
                    if node.tag == "id_info":
                        for child in node:
                            if child is not None:
                                if child.tag == "nct_id":
                                    index = child.text
                    if node.tag == "eligibility":
                        if node[0][0] is not None and node[0][0].text is not None:
                            eligibility = node[0][0].text
#                             print index
#                             print node[0][0].text
                    if eligibility is not None:
                        d[index] = eligibility
    except:
        d[index] = "N/A"
        continue

In [3]:
df_nct_el = pd.DataFrame(list(d.items()), columns=['NCT Number', 'Eligibility'])

In [5]:
df_nct_el.head()

,NCT Number,Eligibility
0,NCT02325466,\n Inclusion Criteria:\n\n - ...
1,NCT00555217,\n Inclusion Criteria:\n\n - ...
2,NCT03211858,\n Inclusion criteria :\n\n - ...
3,NCT02429258,\n Inclusion Criteria:\n\n - ...
4,NCT01106625,\n Inclusion Criteria:\n\n - ...


## Splitting inclusion and exclusion from eligibility

In [6]:
def get_inclexcl(el):
    el = el.replace("\n", " ")
    el = el.strip()
    inc = re.findall(r'Inclusion\ Criteria(.*?)Exclusion\ Criteria', el)
    ex = re.findall(r'Exclusion\ Criteria(.*)', el)
    if not inc and not ex:
        return(el[0],"")
    if inc and ex:
        # return in str format
        return(inc[0],ex[0])
    if inc and not ex:
        return(inc[0],"")
    if ex and not inc:
        return("",ex[0])

In [7]:
inc_arr = []
excl_arr = []

for i in range(len(df_nct_el)):
    el = df_nct_el['Eligibility'][i]
    incl, excl = get_inclexcl(el)
    inc_arr.append(incl)
    excl_arr.append(excl)


In [8]:
temp1 = np.array(inc_arr)
temp2 = np.array(excl_arr)

In [9]:
temp3 = list(d)

In [10]:
df_nct_incl_excl = pd.DataFrame({'NCT':temp3,'Inclusion':temp1, 'Exclusion':temp2})

In [11]:
df_nct_incl_excl.to_csv("inclusion_exclusion.csv",encoding='utf-8')